# Export

Parse notebook and extract exportable code cells into corresponding TS modules
(directives shamelessly copied from `nbdev`)

In [38]:
//| export

import path from "node:path";
import { loadNb } from "jurassic/notebooks.ts";
import { getNotebooksToProcess } from "jurassic/utils.ts";
import type { Config } from "jurassic/config.ts";
import type { Cell } from "jurassic/notebooks.ts";

Helpers for determining if a given line in a cell is a directive. Directives
look like this:

```ts
//| export
```

In [39]:
//| export

const isDirective = (ln: string): boolean =>
  ln.replaceAll(" ", "").startsWith("//|");

In [40]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("isDirective", () => {
  assertEquals(isDirective("//| export"), true);
  assertEquals(isDirective("const c = 1;"), false);
  assertEquals(isDirective("// | export"), true);
  assertEquals(isDirective("// |    export"), true);
});


isDirective ... ok (0ms)

ok | 1 passed | 0 failed (3ms)


Determine if a given cell is exportable. "Exportable" means that its contents
will end up in corresponding ts module.

In [41]:
//| export

const isCellExportable = (cell: Cell): boolean =>
  cell.cell_type === "code" &&
  cell.source.length > 0 &&
  isDirective(cell.source[0]) &&
  cell.source[0].includes("export");

In [42]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("isCellExportable", () => {
  assertEquals(
    isCellExportable({ cell_type: "code", source: ["//| export\n"] }),
    true,
  );
  assertEquals(
    isCellExportable({ cell_type: "code", source: ["const c = 1;"] }),
    false,
  );
  assertEquals(
    isCellExportable({ cell_type: "code", source: ["//|export\n"] }),
    true,
  );
  assertEquals(
    isCellExportable({
      cell_type: "markdown",
      source: ["# showing //| export\n"],
    }),
    false,
  );
  assertEquals(
    isCellExportable({ cell_type: "code", source: ["//|export"] }),
    true,
  );
});

isCellExportable ... ok (0ms)

ok | 1 passed | 0 failed (0ms)


Process notebook - transfer exportable code from cells into ts module

In [43]:
//| export

const moduleHeader = (moduleName: string): string =>
  `// 🦕 AUTOGENERATED! DO NOT EDIT! File to edit: ${moduleName}\n\n`;

const processNb = async (
  nbPath: string,
  moduleName: string,
): Promise<string> => {
  const nb = await loadNb(nbPath);
  // we only need exportable cells
  const exportCells = nb.cells.filter((cell) => isCellExportable(cell));
  return exportCells.reduce(
    // get rid of directives, we want code only
    (acc, cell) => acc + cell.source.filter((s) => !isDirective(s)).join(""),
    moduleHeader(moduleName),
  );
};

In [44]:
await Deno.jupyter.display(
  {
    "text/markdown": "```ts\n\n" +
      (await processNb(path.resolve("./export.ipynb"), "export.ipynb")) +
      "\n```",
  },
  { raw: true },
);

```ts

// 🦕 AUTOGENERATED! DO NOT EDIT! File to edit: export.ipynb


import path from "node:path";
import { loadNb } from "jurassic/notebooks.ts";
import { getNotebooksToProcess  } from "jurassic/utils.ts";
import type { Config } from "jurassic/config.ts";
import type { Cell } from "jurassic/notebooks.ts";
const isDirective = (ln: string): boolean =>
  ln.replaceAll(" ", "").startsWith("//|");
const isCellExportable = (cell: Cell): boolean =>
  cell.cell_type === "code" &&
  cell.source.length > 0 &&
  isDirective(cell.source[0]) &&
  cell.source[0].includes("export");
const moduleHeader = (moduleName: string): string =>
  `// 🦕 AUTOGENERATED! DO NOT EDIT! File to edit: ${moduleName}\n\n`;

const processNb = async (
  nbPath: string,
  moduleName: string,
): Promise<string> => {
  const nb = await loadNb(nbPath);
  // we only need exportable cells
  const exportCells = nb.cells.filter((cell) => isCellExportable(cell));
  return exportCells.reduce(
    // get rid of directives, we want code only
    (acc, cell) => acc + cell.source.filter((s) => !isDirective(s)).join(""),
    moduleHeader(moduleName),
  );
};
export const exportNb = async (
  notebookPath: string,
  config: Config,
): Promise<void> => {
  const notebooksToProcess: string[] = getNotebooksToProcess(
    notebookPath,
    config.nbsPath,
  );

  try {
    await Deno.stat(config.outputPath);
    await Deno.remove(config.outputPath, { recursive: true });
  } catch {
    // noop
  }

  // let's go through all notebooks and process them one by one
  for (const notebook of notebooksToProcess) {
    // output module is the same as the input notebook, but with .ts extension
    const outputFile = notebook.replace(".ipynb", ".ts");
    // make sure we preserve subdirectories if any
    const outputDir = path.join(config.outputPath, path.dirname(outputFile));
    await Deno.mkdir(outputDir, { recursive: true });
    await Deno.writeTextFile(
      path.join(config.outputPath, outputFile),
      await processNb(path.resolve(config.nbsPath, notebook), notebook),
    );
  }
};
```

Main export functionality. `exportNb` should work on both individual notebooks
and directories containing notebooks and subdirectories containing more
notebooks 🕳. `notebookPath` is relative to `config.nbsPath`

In [45]:
// | export

export const exportNb = async (
  notebookPath: string,
  config: Config,
): Promise<void> => {
  const notebooksToProcess: string[] = getNotebooksToProcess(
    notebookPath,
    config.nbsPath,
  );

  try {
    await Deno.stat(config.outputPath);
    await Deno.remove(config.outputPath, { recursive: true });
  } catch {
    // noop
  }

  // let's go through all notebooks and process them one by one
  for (const notebook of notebooksToProcess) {
    // output module is the same as the input notebook, but with .ts extension
    const outputFile = notebook.replace(".ipynb", ".ts");
    // make sure we preserve subdirectories if any
    const outputDir = path.join(config.outputPath, path.dirname(outputFile));
    await Deno.mkdir(outputDir, { recursive: true });
    await Deno.writeTextFile(
      path.join(config.outputPath, outputFile),
      await processNb(path.resolve(config.nbsPath, notebook), notebook),
    );
  }
};

## Tests

Let's test export functionality

In [46]:
import { assert } from "jsr:@std/assert";
import { getTestConfig } from "jurassic/config.ts";
import { dirListing } from "jurassic/utils.ts";

Deno.test("export", async (t) => {
  // set things up, let's recreate mini project structure inside a temp dir
  const td = await Deno.makeTempDir({});

  // recreate nbs dire in temp dir and copy notebooks there
  await Deno.mkdir(`${td}/nbs`),
    Deno.copyFileSync("./export.ipynb", `${td}/nbs/export.ipynb`);
  // recreate submodule directory and copy hello.ipynb to it
  await Deno.mkdir(`${td}/nbs/submodule`),
    Deno.copyFileSync(
      "./submodule/hello.ipynb",
      `${td}/nbs/submodule/hello.ipynb`,
    );

  await t.step("test export", async () => {
    await exportNb("./", getTestConfig(td));

    // make sure output modules are created
    const exportContent = await Deno.readTextFile(`${td}/jurassic/export.ts`);
    const submoduleExportContent = await Deno.readTextFile(
      `${td}/jurassic/submodule/hello.ts`,
    );

    // spot check content inside the output modules
    assert(exportContent.includes("export const exportNb"));
    assert(submoduleExportContent.includes("export const foo"));

    // pretty print temp directory structure
    await Deno.jupyter.display({
      "text/markdown": "```md\n" + (await dirListing(td)) + "\n```",
    }, { raw: true });
  });
});

export ...
  test export ...

```md
- nbs
  - submodule
    - hello.ipynb
  - export.ipynb
- jurassic
  - submodule
    - hello.ts
  - export.ts

```

 ok (2ms)
export ... ok (5ms)

ok | 1 passed (1 step) | 0 failed (5ms)
